<a href="https://colab.research.google.com/github/bhavyajethi/Deep-Learning-practice/blob/main/keras_tuner_hyperparamter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y keras-tuner
!pip install keras-tuner

import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
import keras_tuner
import keras



Found existing installation: keras-tuner 1.4.7
Uninstalling keras-tuner-1.4.7:
  Successfully uninstalled keras-tuner-1.4.7
  Using cached keras_tuner-1.4.7-py3-none-any.whl.metadata (5.4 kB)
Using cached keras_tuner-1.4.7-py3-none-any.whl (129 kB)


<ipython-input-1-580281153>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
df = pd.read_csv('/content/Churn_Modelling.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [20]:
print(df.dtypes)

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object


In [26]:
categorical_col = df.select_dtypes(include=['object','category']).columns

if len(categorical_col) > 0:
  df = pd.get_dummies(columns=categorical_col, data=df, drop_first=True)

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

# Hyperparameters required
# 1) no. of hidden layers required
# 2) no. of neurons in hidden layers
# 3) learning rate

In [27]:
def build_model(hp):
  # Use tf.keras for Sequential
  model = tf.keras.Sequential()
  for i in range(hp.Int('num_layers', 2, 20)):
    model.add(layers.Dense(units=hp.Int('units_'+str(i),
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu'))
  # Move the final output layer and compile outside the loop
  model.add(layers.Dense(1, activation='linear'))
  # Use tf.keras for optimizer
  model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'sgd']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
  return model

In [32]:
tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='hyperparameter tuning',
    project_name='Churn Modelling')

Reloading Tuner from hyperparameter tuning/Churn Modelling/tuner0.json


In [33]:
tuner.search_space_summary()

Search space summary
Default search space size: 22
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'sgd'], 'ordered': False}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_6 (Int)


In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [35]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Trial 5 Complete [00h 07m 44s]
val_accuracy: 0.5534999966621399

Best val_accuracy So Far: 0.5534999966621399
Total elapsed time: 00h 31m 23s


In [36]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

In [37]:
# Building the model with the best hyperparameters
model = tuner.hypermodel.build(best_hp)
# Training the final model
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 47ms/step - accuracy: 0.5400 - loss: 7.3342 - val_accuracy: 0.5535 - val_loss: 7.1183
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 20s 43ms/step - accuracy: 0.5447 - loss: 7.2583 - val_accuracy: 0.5535 - val_loss: 7.1183
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 20s 41ms/step - accuracy: 0.5421 - loss: 7.3008 - val_accuracy: 0.5535 - val_loss: 7.1183
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.5405 - loss: 7.3255 - val_accuracy: 0.5535 - val_loss: 7.1183
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - accuracy: 0.5444 - loss: 7.2634 - val_accuracy: 0.5535 - val_loss: 7.1183
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.5427 - loss: 7.2906 - val_accuracy: 0.5535 - val_loss: 7.1183
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 33ms/step - accuracy: 0.5459 - loss: 7.2399 - val_accuracy: 0.5535 - val_loss: 7.1183
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step - accuracy: 0.5368 - loss: 7.3849 - v

In [39]:
tuner.results_summary()

Results summary
Results in hyperparameter tuning/Churn Modelling
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 4 summary
Hyperparameters:
num_layers: 18
units_0: 64
units_1: 288
optimizer: adam
units_2: 320
units_3: 64
units_4: 416
units_5: 160
units_6: 480
units_7: 448
units_8: 192
units_9: 192
units_10: 128
units_11: 320
units_12: 352
units_13: 512
units_14: 224
units_15: 160
units_16: 480
units_17: 416
units_18: 32
units_19: 32
Score: 0.5534999966621399

Trial 2 summary
Hyperparameters:
num_layers: 5
units_0: 480
units_1: 352
optimizer: adam
units_2: 192
units_3: 288
units_4: 448
units_5: 224
units_6: 512
units_7: 512
units_8: 224
units_9: 416
units_10: 480
units_11: 512
units_12: 480
units_13: 64
units_14: 480
units_15: 128
units_16: 64
units_17: 512
units_18: 352
units_19: 352
Score: 0.5178333322207133

Trial 3 summary
Hyperparameters:
num_layers: 8
units_0: 352
units_1: 160
optimizer: adam
units_2: 352
units_3: 160
units_4: 320
units_5: 224
units_6